## Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def reconstruct(pix_str, size=(48,48)):
    pix_arr = np.array(map(int, pix_str.split()))
    return pix_arr.reshape(size)

def save_data(X_train, X_test, y_train, y_test, fname='', folder='data/'):
    np.save(folder + 'X_train' + fname, X_train)
    np.save(folder + 'y_train' + fname, y_train)
    np.save(folder + 'X_test' + fname, X_test)
    np.save(folder + 'y_test' + fname, y_test)

def preprocess_data(filepath='data/fer2013.csv', split_ratio=0.1):
    
    print 'Reading csv into pandas dataframe'
    df = pd.read_csv(filepath)
    
    print 'Dropping Usage column'
    _Y = df.emotion.replace(0,1)
    _X = df.pixels
    frames = [_X, _Y]
    XY = pd.concat(frames, axis=1)
    
    print ' Unique values',_Y.unique()

    print 'Randomly shuffling the data'
    XY = XY.reindex(np.random.permutation(XY.index))
    
    print 'Reshaping 1x(48*48) array into a 48X48 2D matrix'
    XY['pixels'] = XY.pixels.apply(lambda x: reconstruct(x))

    print 'Converting Dataframe into numpy array'
    X = np.array([mat for mat in XY.pixels])
    X = X.reshape(-1, 1, X.shape[1], X.shape[2])
    Y = np.array([mat for mat in XY.emotion])

    print 'Generating train test splits'
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split_ratio)

    print 'Saving the files'
    save_data(X_train, X_test, y_train, y_test, fname='', folder='data/')

print 'Preprocessing begins'
preprocess_data(filepath='data/fer2013.csv', split_ratio=0.1)
print 'Done!'

## Deep Learning

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, RMSprop
from keras.utils.np_utils import to_categorical

import numpy as np
import sys

In [ ]:
# import dataset:
X_fname = 'data/X_train.npy'
y_fname = 'data/y_train.npy'
X_train = np.load(X_fname)
y_train = np.load(y_fname)
X_train = X_train.astype('float32')
y_train = np.delete(to_categorical(y_train), 0, 1)

In [ ]:
# params:
batch_size = 128
nb_epoch = 2   # originally trained for 100 epochs

# setup info:
print '\n------------------------ Training Parameters -------------------------'
print '                     X_train shape: ', X_train.shape # (n_sample, 1, 48, 48)
print '                     y_train shape: ', y_train.shape # (n_sample, n_categories)
print '                          img size: ', X_train.shape[2],'x', X_train.shape[3]
print '                        batch size: ', batch_size
print '                          nb_epoch: ', nb_epoch

In [ ]:
# model architecture:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu',
                        input_shape=(1, X_train.shape[2], X_train.shape[3])))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(32, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(64, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(Convolution2D(128, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), dim_ordering="th"))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# optimizer:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
print 'Training....'
model.fit(X_train, y_train, nb_epoch=nb_epoch, batch_size=batch_size,
          validation_split=0.1, shuffle=True, verbose=1)

In [ ]:
# model result:
loss_and_metrics = model.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print 'Done!'
print 'Loss: ', loss_and_metrics[0]
print ' Acc: ', loss_and_metrics[1]

In [ ]:
model.save('saved_model/saved_model.h5')

In [ ]:
from keras.models import load_model
model2 = load_model('saved_model/saved_model.h5')

In [ ]:
# model result:
loss_and_metrics = model2.evaluate(X_train, y_train, batch_size=batch_size, verbose=1)
print 'Done!'
print 'Loss: ', loss_and_metrics[0]
print ' Acc: ', loss_and_metrics[1]